# **1. Shallow Learning**

In [3]:
import numpy as np
import pickle
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import numpy as np
import pandas as pd
import torch
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from gensim.models import Word2Vec
from tensorflow.keras.utils import to_categorical
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [4]:
# Cargamos dataset tokenizado
df_train = pd.read_pickle("data/data_clean/train_tokenized.pkl")

print("Clases en bias:", df_train["bias"].value_counts())

os.makedirs("data/models", exist_ok=True)

# Usamos la columna correcta
df_train["text_joined"] = df_train["tokens"].apply(lambda x: " ".join(x))

# TF-IDF optimizado
tfidf_vectorizer = TfidfVectorizer(
    max_features=3000,
    stop_words="english",
    ngram_range=(1, 2)
)

X_tfidf = tfidf_vectorizer.fit_transform(df_train["text_joined"])
y = df_train["bias"].values

# Train/val
X_tr, X_val, y_tr, y_val = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42
)


# Entrenar modelos
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, n_jobs=-1),
    "LinearSVC": LinearSVC(),
    "Random Forest": RandomForestClassifier(n_estimators=150, n_jobs=-1),
    "XGBoost": XGBClassifier(
        n_estimators=200,
        eval_metric="mlogloss",
        tree_method="hist",
        n_jobs=-1
    )
}

results = {}

print("\nEntrenando modelos... (mucho más rápido ahora)\n")

for name, model in models.items():
    print(f"Entrenando {name}...")
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_val)
    results[name] = {
        "Accuracy": accuracy_score(y_val, y_pred),
        "Macro-F1": f1_score(y_val, y_pred, average="macro")
    }
    pickle.dump(model, open(f"data/models/{name.replace(' ', '_').lower()}.pkl", "wb"))

# Guardamos TF-IDF
pickle.dump(tfidf_vectorizer, open("data/features/tfidf_vectorizer.pkl", "wb"))

# Resultados
results_df = pd.DataFrame(results).T
print("\nResultados comparativos Shallow Learning:\n")
print(results_df)


Clases en bias: bias
2    10240
0     9750
1     7988
Name: count, dtype: int64

Entrenando modelos... (mucho más rápido ahora)

Entrenando Logistic Regression...
Entrenando LinearSVC...
Entrenando Random Forest...
Entrenando XGBoost...

Resultados comparativos Shallow Learning:

                     Accuracy  Macro-F1
Logistic Regression  0.696926  0.694518
LinearSVC            0.693531  0.690891
Random Forest        0.694425  0.690053
XGBoost              0.743924  0.743006


Para evaluar los modelos hemos usado Accuracy y Macro-F1, métricas adecuadas para problemas de clasificación con clases desbalanceadas.
Los cuatro modelos presentan resultados parecidos. Sin embargo, XGBoost ofrece el mejor rendimiento, mostrando que puede capturar patrones complejos del sesgo ideológico mejor que modelos lineales o Random Forest.
Por otro lado, los modelos lineales funcionan razonablemente bien, lo que indica que el sesgo tiene señales lineales claras en los términos más frecuentes.
Elegimos estos cuatro modelos (Logistic Regression, LinearSVC, Random Forest y XGBoost) para cubrir tanto enfoques lineales como basados en árboles, y utilizar el dataset tokenizado con TF-IDF para representar el texto en forma dispersa, adecuada para Shallow Learning.

# **3. Modelos Deep**


En esta parte nos vamos a enfocar en la clasificación del sesgo. Los motivos son los siguientes:

1- Detectar la orientación política de una noticia es la tarea principal del proyecto. Además, esta tarea permite evaluar cómo los modelos y embeddings capturan matices semánticos y patrones discursivos.
2- Una vez concluida la clasificación de sesgo, se puede reutilizar la pipeline para las demás tareas de clasificación (medio y temática).
3- La columna bias presenta un número moderadamente equilibrado de ejemplos por clase, lo que permite ajustar la arquitectura y los hiperparámetros de manera controlada antes de afrontar tareas más complejas.

Las razones de haber elegido las combinaciones de embeddings y arquitecturas de redes neuronales para abordar la clasificación del sesgo ideológicoson las siguientes:

1- Comparación de embeddings no contextuales y contextuales:
    -Los embeddings no contextuales como, Word2Vecy y FastText, permiten capturar relaciones semánticas entre palabras de manera estática.
    -Los embeddings contextuales como, Sentence Transformers y BERT, capturan el significado de las palabras según su contexto en la frase, lo que es clave para detectar matices ideológicos más complejos.

2- Exploración de diferentes estrategias de embeddings:
    -Word2Vec congelado: usar embeddings preentrenados sin actualizar durante el entrenamiento, para evaluar la capacidad de vectores fijos.
    -Word2Vec fine-tune: ajustar los vectores durante el entrenamiento para adaptarlos al corpus específico.
    -Word2Vec “from scratch”: entrenar desde cero sobre el dataset, para capturar patrones propios del corpus.
    -Para los embeddings contextuales, se compara Sentence Transformers preentrenado frente a BERT, con fine-tuning parcial o total según la arquitectura de la red.

3- Elección de arquitecturas de redes neuronales:
    -Redes totalmente conectadas (Dense): adecuadas para embeddings agregados o promedio de vectores de palabras.
    -Redes recurrentes (LSTM/GRU): capturan secuencias y dependencias entre palabras, esenciales para comprender el flujo discursivo en los artículos.
    -CNN para texto: permiten identificar patrones locales de n-gramas que son relevantes en la clasificación de sesgo.

4- Razonamiento general:
    -Combinar diferentes tipos de embeddings y arquitecturas permite evaluar cuál representa mejor la información semántica y estilística para cada tarea.
    -Esta estrategia también permite analizar cómo el fine-tuning de embeddings impacta en la capacidad del modelo de captar señales ideológicas, frente a vectores preentrenados fijos.

In [5]:
# Cargamos el dataset tokenizado
df_train = pd.read_pickle("data/data_clean/train_tokenized.pkl")
y = df_train["bias"].values

# Codificamos los labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Split train/validation
X_tr_text, X_val_text, y_tr, y_val = train_test_split(
    df_train["tokens"], y_cat, test_size=0.2, random_state=42
)

# Cargamos el Word2Vec preentrenado de la anterior entrega
w2v_model = Word2Vec.load("data/embeddings/word2vec.model")
embedding_dim = w2v_model.vector_size

# Creamos el vocabulario e índices
word_index = {word: i+1 for i, word in enumerate(w2v_model.wv.index_to_key)}
vocab_size = len(word_index) + 1  # +1 para padding

# Convertimos los tokens a índices
def tokens_to_indices(tokens, word_index):
    return [word_index[t] for t in tokens if t in word_index]

X_tr_idx = [tokens_to_indices(t, word_index) for t in X_tr_text]
X_val_idx = [tokens_to_indices(t, word_index) for t in X_val_text]

# Aplciamos padding
max_seq_len = 200
X_tr_pad = pad_sequences(X_tr_idx, maxlen=max_seq_len, padding='post')
X_val_pad = pad_sequences(X_val_idx, maxlen=max_seq_len, padding='post')

# Creamos la matriz de embedding 
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    embedding_matrix[i] = w2v_model.wv[word]

# Definimos y entrenaos los modelos 

def build_rnn(model_type='LSTM'):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=max_seq_len,
                        trainable=True))  # Fine-tune embeddings
    if model_type == 'LSTM':
        model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    elif model_type == 'GRU':
        model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# LSTM
lstm_model = build_rnn('LSTM')
lstm_history = lstm_model.fit(X_tr_pad, y_tr,
                              validation_data=(X_val_pad, y_val),
                              epochs=10,
                              batch_size=64)

# GRU
gru_model = build_rnn('GRU')
gru_history = gru_model.fit(X_tr_pad, y_tr,
                            validation_data=(X_val_pad, y_val),
                            epochs=10,
                            batch_size=64)

# Evaluamos los modelos
from sklearn.metrics import accuracy_score, f1_score

# Predicciones
y_pred_lstm = lstm_model.predict(X_val_pad, batch_size=64)
y_pred_gru = gru_model.predict(X_val_pad, batch_size=64)

y_pred_lstm_labels = np.argmax(y_pred_lstm, axis=1)
y_pred_gru_labels = np.argmax(y_pred_gru, axis=1)
y_val_labels = np.argmax(y_val, axis=1)

# Métricas
results = {
    'LSTM': {
        'Accuracy': accuracy_score(y_val_labels, y_pred_lstm_labels),
        'Macro-F1': f1_score(y_val_labels, y_pred_lstm_labels, average='macro')
    },
    'GRU': {
        'Accuracy': accuracy_score(y_val_labels, y_pred_gru_labels),
        'Macro-F1': f1_score(y_val_labels, y_pred_gru_labels, average='macro')
    }
}

results_df = pd.DataFrame(results).T
print("Resultados:")
print(results_df)


c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 116s 322ms/step - accuracy: 0.4257 - loss: 1.0562 - val_accuracy: 0.4750 - val_loss: 1.0236
Epoch 2/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 109s 311ms/step - accuracy: 0.5227 - loss: 0.9537 - val_accuracy: 0.4959 - val_loss: 0.9828
Epoch 3/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 119s 340ms/step - accuracy: 0.5904 - loss: 0.8587 - val_accuracy: 0.5234 - val_loss: 0.9534
Epoch 4/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 150s 428ms/step - accuracy: 0.6674 - loss: 0.7404 - val_accuracy: 0.5173 - val_loss: 1.0088
Epoch 5/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 131s 373ms/step - accuracy: 0.7374 - loss: 0.6141 - val_accuracy: 0.5116 - val_loss: 1.0896
Epoch 6/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 182s 521ms/step - accuracy: 0.8024 - loss: 0.4791 - val_accuracy: 0.5046 - val_loss: 1.2833
Epoch 7/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 357s 1s/step - accuracy: 0.8611 - loss: 0.3566 - val_accuracy: 0.5095 - val_loss: 1.3578
Epoch 8/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 364s 1s/step - accuracy: 0.8989 - loss:

-Rendimiento general:
   -Ambos modelos muestran resultados muy similares, con valores en torno al 53–54%, tanto en Accuracy como en Macro-F1. Esto indica que:
       -Los dos modelos capturan de forma parecida los patrones secuenciales del sesgo ideológico.
       -No existe una ventaja clara de ninguno de los dos modelos neuronales en este dataset.
-Interpretación:
    -El rendimiento indica que el sesgo ideológico es una tarea difícil incluso para modelos neuronales. 
    -Puede que los textos no tengan suficiente señal secuencial para que LSTM/GRU destaquen claramente.
-Conclusión:
    -Ambos modelos presentan un rendimiento equivalente, pero al ser  ligeramente superior, hemos decidido usar LSTM  como baseline de deep learning. Sin embargo, estas arquitecturas probablemente no capturan matices ideológicos complejos, por lo que se es necesario explorar  modelos más potentes como BERT o RoBERTa.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from sklearn.metrics import accuracy_score, f1_score

# -----------------------------
#  Cargar dataset tokenizado
# -----------------------------
df = pd.read_pickle("data/data_clean/train_tokenized.pkl")

# Crear columna text_joined a partir de tokens
df["text_joined"] = df["tokens"].apply(lambda x: " ".join(x))

texts = df["text_joined"].astype(str).tolist()
labels = df["bias"].tolist()

# -----------------------------
#  Tokenización y secuencias
# -----------------------------
vocab_size = 20000
maxlen = 100
embedding_dim = 100

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts)
X_seq = tokenizer.texts_to_sequences(texts)
X = pad_sequences(X_seq, maxlen=maxlen)

# Codificación de etiquetas
le = LabelEncoder()
y = le.fit_transform(labels)

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -----------------------------
#  Modelos LSTM y GRU
# -----------------------------
def build_lstm_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=maxlen, trainable=False),
        LSTM(128, return_sequences=False),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(len(np.unique(y)), activation='softmax')
    ])
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"]
    )
    return model

def build_gru_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=maxlen, trainable=False),
        GRU(128, return_sequences=False),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(len(np.unique(y)), activation='softmax')
    ])
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"]
    )
    return model

# -----------------------------
#  Entrenamiento LSTM
# -----------------------------
lstm_model = build_lstm_model()
history_lstm = lstm_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=64
)

# -----------------------------
#  Entrenamiento GRU
# -----------------------------
gru_model = build_gru_model()
history_gru = gru_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=64
)

# -----------------------------
#  Evaluación
# -----------------------------
def evaluate(model):
    preds = np.argmax(model.predict(X_val), axis=1)
    return accuracy_score(y_val, preds), f1_score(y_val, preds, average="macro")

acc_lstm, f1_lstm = evaluate(lstm_model)
acc_gru, f1_gru = evaluate(gru_model)

print("\nResultados Embedding Random (NO fine-tuneado)")
print("LSTM → Accuracy:", acc_lstm, "Macro-F1:", f1_lstm)
print("GRU  → Accuracy:", acc_gru, "Macro-F1:", f1_gru)


KeyError: 'text_joined'

# **5. Tabla Comaprativa de Resultados**

# **6. Interpretabilidad**